In [3]:
import logging
import time
import threading
import os
import pyautogui
import boto3
from datetime import datetime
from pynput import mouse, keyboard
from pystray import Icon, MenuItem, Menu
from PIL import Image, ImageDraw, ImageOps
from botocore.exceptions import NoCredentialsError, PartialCredentialsError, EndpointConnectionError
import tkinter as tk
from tkinter import messagebox
import requests
import io  # For handling image uploads in memory
import psutil  # For battery status monitoring

# Configure logging for activity tracking (log to activity_log.txt)
logging.basicConfig(filename="activity_log.txt", level=logging.INFO)

# Global variables for tracking mouse and keyboard activity
is_active = False
last_mouse_pos = None
last_mouse_time = None
last_keypress_time = None

# Thresholds for detecting suspicious activity
MOUSE_SPEED_THRESHOLD = 500  # pixels per second (for fast mouse movements)
KEYPRESS_THRESHOLD = 0.05  # seconds (for rapid key presses)
STRAIGHT_LINE_THRESHOLD = 10  # pixels for straight-line movement detection

# Function to create an icon for the system tray (taskbar)
def create_icon_image():
    width, height = 64, 64
    image = Image.new("RGB", (width, height), (255, 255, 255))  # Create a white image
    draw = ImageDraw.Draw(image)
    draw.rectangle([0, 0, width, height], fill=(0, 128, 255))  # Blue square icon
    return image

# Function to log activity (mouse movement, clicks, key presses)
def log_activity(action_type, action_detail, is_suspicious=False):
    timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
    log_entry = f"{timestamp} - {action_type}: {action_detail}"
    if is_suspicious:
        log_entry += " [Suspicious Activity Detected]"
    logging.info(log_entry)  # Write to the activity log file
    print(log_entry)  # Also print to console
    upload_log_to_s3()  # Upload log to S3 after every log entry

# Mouse activity tracking
def on_move(x, y):
    global is_active, last_mouse_pos, last_mouse_time
    if last_mouse_pos is not None and last_mouse_time is not None:
        time_diff = time.time() - last_mouse_time
        distance = ((x - last_mouse_pos[0]) ** 2 + (y - last_mouse_pos[1]) ** 2) ** 0.5
        speed = distance / time_diff  # Speed in pixels per second
        
        # Check for suspiciously fast mouse movement
        if speed > MOUSE_SPEED_THRESHOLD:
            log_activity("Mouse Move", f"Suspicious fast mouse movement detected at {x}, {y}", is_suspicious=True)
            is_active = True
        # Check if the movement is too linear (straight-line movement)
        elif abs(x - last_mouse_pos[0]) < STRAIGHT_LINE_THRESHOLD and abs(y - last_mouse_pos[1]) < STRAIGHT_LINE_THRESHOLD:
            log_activity("Mouse Move", f"Suspicious straight-line movement detected at {x}, {y}", is_suspicious=True)
            is_active = True
        else:
            log_activity("Mouse Move", f"Mouse moved to {x}, {y}")
            is_active = True

    last_mouse_pos = (x, y)
    last_mouse_time = time.time()

def on_click(x, y, button, pressed):
    if pressed:
        log_activity("Mouse Click", f"Mouse clicked at {x}, {y}")

def on_scroll(x, y, dx, dy):
    log_activity("Mouse Scroll", f"Mouse scrolled at {x}, {y}, delta: ({dx}, {dy})")

# Keyboard activity tracking
def on_press(key):
    global last_keypress_time
    current_time = time.time()

    # Check for rapid key presses
    if last_keypress_time is not None:
        time_diff = current_time - last_keypress_time
        if time_diff < KEYPRESS_THRESHOLD:
            log_activity("Key Pressed", f"Suspicious rapid key press detected: {key}", is_suspicious=True)

    last_keypress_time = current_time
    log_activity("Key Pressed", f"Key {key} pressed")

# Quit system tray icon
def on_quit(icon, item):
    icon.stop()

# Start listeners for mouse and keyboard events
def start_activity_listeners():
    mouse_listener = mouse.Listener(on_move=on_move, on_click=on_click, on_scroll=on_scroll)
    mouse_listener.start()

    keyboard_listener = keyboard.Listener(on_press=on_press)
    keyboard_listener.start()

# System tray menu options
menu = Menu(MenuItem('Quit', on_quit))  # Option to quit the application
icon = Icon("Activity Tracker", create_icon_image(), menu=menu)  # System tray icon

# ActivityTracker class for managing screenshots and uploads
class ActivityTracker:
    def __init__(self, screenshot_interval=60, s3_bucket_name=None, s3_region='us-east-1'):
        self.screenshot_interval = screenshot_interval  # Interval in seconds for screenshots
        self.s3_bucket_name = s3_bucket_name  # S3 bucket to upload files
        self.s3_region = s3_region  # AWS region for S3
        self.s3_client = boto3.client('s3', region_name=self.s3_region)  # S3 client
        self.last_screenshot_time = time.time()  # Last screenshot time
        self.upload_queue = []  # Queue for files if offline
        self.is_tracking = True  # Flag to control tracking status

    # Capture and compress a screenshot
    def capture_and_compress_screenshot(self):
        if not self.is_tracking:
            return  # Skip if tracking is paused
        
        current_time = time.time()
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        screenshot = pyautogui.screenshot()  # Take screenshot
        
        # Compress the image using Pillow
        compressed_screenshot = self.compress_image(screenshot)
        
        # Save to memory using io.BytesIO
        img_byte_arr = io.BytesIO()
        compressed_screenshot.save(img_byte_arr, format='PNG')
        img_byte_arr.seek(0)  # Rewind to the beginning of the image
        
        # Upload the screenshot to S3 and delete local screenshot if upload is successful
        self.upload_file_to_s3(img_byte_arr, f"screenshots/screenshot_{timestamp}.png")

        self.last_screenshot_time = current_time  # Update the last screenshot time

    # Compress the image for efficient upload
    def compress_image(self, image: Image):
        max_width, max_height = 1024, 1024  # Resize constraints
        image.thumbnail((max_width, max_height), Image.ANTIALIAS)
        image = ImageOps.exif_transpose(image)  # Correct any rotation based on EXIF
        return image

    # Upload the screenshot to S3 and delete the local file
    def upload_file_to_s3(self, file_data, s3_key):
        if not self.s3_bucket_name:
            print("S3 bucket name is missing.")
            return
        
        if not self.is_connected_to_internet():
            print("No internet. Queuing upload.")
            self.upload_queue.append((file_data, s3_key))  # Queue file for later
            return

        try:
            self.s3_client.upload_fileobj(file_data, self.s3_bucket_name, s3_key)  # Upload file directly from memory
            print(f"Uploaded screenshot to S3: {s3_key}")
            self.delete_local_screenshot(s3_key)  # Delete the local file after upload
        except (NoCredentialsError, PartialCredentialsError) as e:
            print(f"Credentials error: {e}")
        except EndpointConnectionError as e:
            print(f"Network error: {e}")
        except Exception as e:
            print(f"Error uploading to S3: {e}")

    # Delete the local screenshot after successful upload
    def delete_local_screenshot(self, screenshot_key):
        # Here we assume screenshots are stored in memory, but if you were saving to disk:
        screenshot_file = f"screenshots/{screenshot_key}"
        if os.path.exists(screenshot_file):
            os.remove(screenshot_file)
            print(f"Deleted local screenshot: {screenshot_file}")

    # Upload activity log to S3
    def upload_log_to_s3(self):
        log_file = "activity_log.txt"
        if os.path.exists(log_file):
            if not self.is_connected_to_internet():
                print("No internet. Cannot upload log.")
                return  # Skip upload if offline

            try:
                s3_object_key = f"logs/{log_file}"
                self.s3_client.upload_file(log_file, self.s3_bucket_name, s3_object_key)
                print(f"Uploaded log file to S3: {s3_object_key}")
            except Exception as e:
                print(f"Error uploading log file to S3: {e}")

    # Check if there is an active internet connection
    def is_connected_to_internet(self):
        try:
            requests.get("https://www.google.com", timeout=5)  # Try reaching Google
            return True
        except (requests.ConnectionError, requests.Timeout):
            return False

    # Periodically capture screenshots
    def capture_screenshots_periodically(self):
        while True:
            if self.is_tracking and time.time() - self.last_screenshot_time >= self.screenshot_interval:
                self.capture_and_compress_screenshot()
            time.sleep(self.screenshot_interval)

    # Check battery status and pause tracking if battery is low
    def check_battery_and_manage_tracking(self):
        while True:
            battery = psutil.sensors_battery()
            if battery:
                battery_percentage = battery.percent
                if battery_percentage < 20:
                    print(f"Low battery ({battery_percentage}%). Pausing tracking.")
                    self.is_tracking = False
                elif battery_percentage > 40:
                    if not self.is_tracking:
                        print(f"Battery sufficient ({battery_percentage}%). Resuming tracking.")
                        self.is_tracking = True
            time.sleep(60)

# Configuration window for user settings
class ConfigWindow:
    def __init__(self, root, tracker):
        self.tracker = tracker
        self.root = root
        self.root.title("Activity Tracker Settings")
        self.create_widgets()

    def create_widgets(self):
        self.screenshot_interval_label = tk.Label(self.root, text="Screenshot Interval (seconds):")
        self.screenshot_interval_label.grid(row=0, column=0, padx=10, pady=10)
        
        self.screenshot_interval_entry = tk.Entry(self.root)
        self.screenshot_interval_entry.grid(row=0, column=1, padx=10, pady=10)
        self.screenshot_interval_entry.insert(0, str(self.tracker.screenshot_interval))  # Default value

        self.enable_screenshot_var = tk.BooleanVar(value=True)  # Default to enabled
        self.enable_screenshot_check = tk.Checkbutton(self.root, text="Enable Screenshot Capture", variable=self.enable_screenshot_var)
        self.enable_screenshot_check.grid(row=1, column=0, columnspan=2, padx=10, pady=10)

        self.save_button = tk.Button(self.root, text="Save Settings", command=self.save_settings)
        self.save_button.grid(row=2, column=0, columnspan=2, padx=10, pady=10)

    def save_settings(self):
        interval = self.screenshot_interval_entry.get()
        try:
            interval = int(interval)
            if interval <= 0:
                raise ValueError("Interval must be a positive integer.")
            enable_screenshot = self.enable_screenshot_var.get()
            self.tracker.screenshot_interval = interval
            messagebox.showinfo("Settings Saved", f"Settings saved: Interval: {interval} seconds\nScreenshot Capture: {'Enabled' if enable_screenshot else 'Disabled'}")
        except ValueError as e:
            messagebox.showerror("Invalid Input", f"Error: {e}")

# Main function to start the system tray application
def start_application():
    tracker = ActivityTracker(screenshot_interval=60, s3_bucket_name="your_s3_bucket_name")
    config_window = tk.Tk()
    ConfigWindow(config_window, tracker)  # Start the config window
    config_window.mainloop()  # Start the GUI

    # Start activity listeners for mouse and keyboard
    start_activity_listeners()

    # Start capturing screenshots periodically
    threading.Thread(target=tracker.capture_screenshots_periodically, daemon=True).start()

    # Start checking battery status and manage tracking based on battery
    threading.Thread(target=tracker.check_battery_and_manage_tracking, daemon=True).start()

    # Start the system tray icon
    icon.run()

# Run the application
if __name__ == "__main__":
    start_application()


2024-11-27 09:52:30 - Mouse Move: Suspicious straight-line movement detected at 398, 92 [Suspicious Activity Detected]


Exception in thread Thread-12:
Traceback (most recent call last):
  File "C:\Users\HAMID SAIFI\anaconda3\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "C:\Users\HAMID SAIFI\anaconda3\lib\threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\HAMID SAIFI\AppData\Local\Temp\ipykernel_12228\1744217949.py", line 207, in capture_screenshots_periodically
  File "C:\Users\HAMID SAIFI\AppData\Local\Temp\ipykernel_12228\1744217949.py", line 131, in capture_and_compress_screenshot
  File "C:\Users\HAMID SAIFI\AppData\Local\Temp\ipykernel_12228\1744217949.py", line 146, in compress_image
AttributeError: module 'PIL.Image' has no attribute 'ANTIALIAS'


2024-11-27 09:53:38 - Key Pressed: Key Key.enter pressed
